# Langchain是什么
Langchain可以被看成一个AI应用的开发框架，它把AI应用中会用到的组件分成以下几类：
- 模型 Models
- 提示词 Prompts
- 索引 Indexes
- Memory
- Chains 
- Agents

Langchain的价值在于，它提供了一层抽象，使得应用开发中的逻辑层和实现层可以解耦，比如模型部分可以使用GPT-3.5、GPT-4.0、GLM平滑替换。这层抽象也使扩展程序变得更简单，可以很轻松的引入新的功能。

## 模型 Models
目前Langchain支持三种模型：
- LLMs(Large Language Models)：接受文本输入，返回文本作为输出
- Chat Models: 本质上还是LLMs，只是输入输出是有一定结构的聊天信息
- Text Embedding Models: 接受文本输入，返回浮点数列表作为输出

## 提示词 Prompts

## 索引 Indexes

## Memory

## Chains

## Agents

In [1]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms.openai import OpenAI
from langchain.chat_models import PromptLayerChatOpenAI

In [2]:
chat = PromptLayerChatOpenAI(pl_tags=["agent-executor"], temperature=0, verbose=True)
agent_executor = create_python_agent(
    llm=chat,
    tool=PythonREPLTool(),
    verbose=True
)

In [3]:
agent_executor.run("Find the roots (zeroes) if the quadratic function 3 * x**2 + 2*x -1")



> Entering new AgentExecutor chain...
I can use the quadratic formula to find the roots of the function.
Action: Python REPL
Action Input: 
```
import math
a = 3
b = 2
c = -1
root1 = (-b + math.sqrt(b**2 - 4*a*c)) / (2*a)
root2 = (-b - math.sqrt(b**2 - 4*a*c)) / (2*a)
print(root1, root2)
```
Observation: 0.3333333333333333 -1.0

Thought:The roots of the quadratic function are 0.3333333333333333 and -1.0
Final Answer: 0.3333333333333333, -1.0

> Finished chain.


'0.3333333333333333, -1.0'

# 其它


## 获取Token消耗

In [3]:
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

def count_tokens(chain, query):
    with get_openai_callback() as cb:
        result = chain.run(query)
        print(f'Spent a total of {cb.total_tokens} tokens, cost ${cb.total_cost}')

    return result

llm = OpenAI(model_name="text-davinci-002", n=2, best_of=2)
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    print(cb)

Tokens Used: 42
	Prompt Tokens: 4
	Completion Tokens: 38
Successful Requests: 1
Total Cost (USD): $0.00084


In [2]:
with get_openai_callback() as cb:
    result = llm("Tell me a joke")
    result2 = llm("Tell me a joke")
    print(cb)

84


In [1]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
with get_openai_callback() as cb:
    response = agent.run("Who is Olivia Wilde's boyfriend? What is his current age raised to the 0.23 power?")
    print(f"Total Tokens: {cb.total_tokens}")
    print(f"Prompt Tokens: {cb.prompt_tokens}")
    print(f"Completion Tokens: {cb.completion_tokens}")
    print(f"Total Cost (USD): ${cb.total_cost}")

ValidationError: 1 validation error for SerpAPIWrapper
__root__
  Did not find serpapi_api_key, please add an environment variable `SERPAPI_API_KEY` which contains it, or pass  `serpapi_api_key` as a named parameter. (type=value_error)

## 结构化输出

In [5]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)

# 告诉他我们生成的内容需要哪些字段，每个字段类型式啥
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# 初始化解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 生成的格式提示符
# {
#	"bad_string": string  // This a poorly formatted user input string
#	"good_string": string  // This is your response, a reformatted response
#}
format_instructions = output_parser.get_format_instructions()

template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# 讲我们的格式描述嵌入到 prompt 中去，告诉 llm 我们需要他输出什么样格式的内容
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

promptValue = prompt.format(user_input="welcom to califonya!")
print(promptValue)
llm_output = llm(promptValue)

# 使用解析器进行解析生成的内容
output_parser.parse(llm_output)


You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\`\`\`json" and "\`\`\`":

```json
{
	"bad_string": string  // This a poorly formatted user input string
	"good_string": string  // This is your response, a reformatted response
}
```

% USER INPUT:
welcom to califonya!

YOUR RESPONSE:



{'bad_string': 'welcom to califonya!', 'good_string': 'Welcome to California!'}

## Using HuggingFace

In [2]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}
Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])
llm = HuggingFaceHub(repo_id="google/flan-t5-xl", model_kwargs={"temperature":0, "max_length":64})
llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"
print(llm_chain.run(question))

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

In [3]:
# 将 HuggingFace 模型直接拉到本地使用
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

model_id = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)
print(local_llm('What is the capital of France? '))


llm_chain = LLMChain(prompt=prompt,  llm=local_llm)
question = "What is the capital of England?"
print(llm_chain.run(question))

/Users/guokun/code/python/p38/lib/python3.8/site-packages/mpmath/ctx_mp_python.py:873: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:
/Users/guokun/code/python/p38/lib/python3.8/site-packages/mpmath/ctx_mp_python.py:967: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if other is 0:


RuntimeError: Failed to import transformers.pipelines because of the following error (look up to see its traceback):
No module named 'opt_einsum'

In [4]:
from huggingface_hub.inference_api import InferenceApi
inference = InferenceApi(repo_id="bert-base-uncased")

print(inference(inputs="The goal of life is [MASK]."))

[{'score': 0.10933294892311096, 'token': 2166, 'token_str': 'life', 'sequence': 'the goal of life is life.'}, {'score': 0.039418820291757584, 'token': 7691, 'token_str': 'survival', 'sequence': 'the goal of life is survival.'}, {'score': 0.03293056786060333, 'token': 2293, 'token_str': 'love', 'sequence': 'the goal of life is love.'}, {'score': 0.030096111819148064, 'token': 4071, 'token_str': 'freedom', 'sequence': 'the goal of life is freedom.'}, {'score': 0.024967176839709282, 'token': 17839, 'token_str': 'simplicity', 'sequence': 'the goal of life is simplicity.'}]
